## Efficiency Metrics Summary

### Overview  
This notebook provides a consolidated analysis of Partizan Mozzart Bet Belgrade’s player performance in the 2022-2023 EuroLeague season, based on six key efficiency metrics:  

- **True Shooting Percentage (TS%)**: Measures shooting efficiency, factoring in 2-point, 3-point, and free-throw shots.  
- **Points per 32 Minutes (PP32)**: Normalizes scoring output to a standard 32-minute playing time.  
- **Assist-to-Turnover Ratio (AST/TO)**: Assesses playmaking efficiency by comparing assists to turnovers.  
- **Usage Rate (USG%)**: Percentage of team possessions a player uses, adjusted for time on court.  
- **Points per Possession (PPP)**: Scoring efficiency per possession used.  
- **Rebound Rate (REB%)**: Percentage of available team rebounds a player secures, adjusted for minutes.

In [1]:
import pandas as pd

file_path = "../data/partizan_2022_cleaned.csv"

df = pd.read_csv(file_path)

In [2]:
# filter
players = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
team = df[df['player'] == 'PARTIZAN MOZZART BET BELGRADE'].copy()

In [3]:
# snippet from true_shooting_percentage.ipynb

players['field_goals_attempted'] = players['two_points_attempted'] + players['three_points_attempted']
players['ts_percentage'] = players['points'] / (2 * (players['field_goals_attempted'] + 0.44 * players['free_throws_attempted']))

# handle division by zero (players with no shot attempts)
players['ts_percentage'] = players['ts_percentage'].fillna(0).replace([float('inf'), -float('inf')], 0)

# average TS% per player
avg_ts = players.groupby('player')['ts_percentage'].mean().sort_values(ascending=False).head(14)


print("Top 5 Players by True Shooting Percentage (TS%):")
print(avg_ts.head())

Top 5 Players by True Shooting Percentage (TS%):
player
Lessort Mathias    0.721486
Exum Dante         0.671916
Punter Kevin       0.623184
Smailagic Alen     0.619905
Leday Zach         0.587875
Name: ts_percentage, dtype: float64


In [4]:
# snippet from points_per_32.ipynb

# calculate Points per 32 Minutes (close to typical starter's minutes)
# formula: (Points / Total Seconds) * 1920 (where 1920 seconds = 32 minutes)
players['points_per_32'] = (players['points'] / players['total_seconds']) * 1920

# handle division by zero (players with 0 seconds)
players['points_per_32'] = players['points_per_32'].fillna(0).replace([float('inf'), -float('inf')], 0)

# average points per 32 minutes per player
avg_points_per_32 = players.groupby('player')['points_per_32'].mean().sort_values(ascending=False).head(14)


print("Top 5 players by Points per 32 Minutes:")
print(avg_points_per_32.head(5))

Top 5 players by Points per 32 Minutes:
player
Punter Kevin        18.341875
Exum Dante          18.070395
Smailagic Alen      17.346076
Lessort Mathias     12.758641
Avramovic Aleksa    12.523459
Name: points_per_32, dtype: float64


In [5]:
# snippet from assist_to_turnover_ratio.ipynb

# calculate Assist-to-Turnover Ratio (AST/TO) per game 
# formula: AST/TO = assists / turnovers or assists if turnovers == 0
players['ast_to_ratio'] = players.apply(
    lambda row: row['assists'] if row['turnovers'] == 0 else row['assists'] / row['turnovers'], 
    axis=1
)

# average AST/TO per player
avg_ast_to = players.groupby('player')['ast_to_ratio'].mean().sort_values(ascending=False).head(14)


print("Top 5 Players by Assist-to-Turnover Ratio per game (AST/TO):")
print(avg_ast_to.head())

Top 5 Players by Assist-to-Turnover Ratio per game (AST/TO):
player
Exum Dante          1.857456
Punter Kevin        1.844144
Nunnally James      1.401316
Madar Yam           1.391026
Avramovic Aleksa    1.201389
Name: ast_to_ratio, dtype: float64


In [6]:
# snippet from usage_rate.ipynb

# calculate Usage Rate (USG%)
# Usage Rate (USG%): Percentage of team possessions used by a player, adjusted for playing time, calculated as:
# formula: USG% = 100 * ((FGA + 0.44 * FTA + TO) / (FGA_team + 0.44 * FTA_team + TO_team)) * (TS / 2400)

players_1 = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
team_1 = df[df['player'] == 'PARTIZAN MOZZART BET BELGRADE'].copy()

# calculate total shot attempts because of merging below
players_1['field_goals_attempted'] = players_1['two_points_attempted'] + players_1['three_points_attempted']
team_1['field_goals_attempted'] = team_1['two_points_attempted'] + team_1['three_points_attempted']

# merge team stats to players_1
players_1 = players_1.merge(team_1[['game', 'field_goals_attempted', 'free_throws_attempted', 'turnovers', 'total_seconds']], 
                            on='game', suffixes=('', '_team')).reset_index(drop=True)

# calculate Usage Rate
game_seconds = 2400  # 40 minutes * 60 seconds
players_1['usage_rate'] = 100 * ((players_1['field_goals_attempted'] + 0.44 * players_1['free_throws_attempted'] + players_1['turnovers']) /
                                 (players_1['field_goals_attempted_team'] + 0.44 * players_1['free_throws_attempted_team'] + players_1['turnovers_team'])) * \
                                (players_1['total_seconds'] / game_seconds)

# handle division by zero (players with no team stats)                              
players_1['usage_rate'] = players_1['usage_rate'].fillna(0).replace([float('inf'), -float('inf')], 0)

# average USG% per player
avg_usg = players_1.groupby('player')['usage_rate'].mean().sort_values(ascending=False)

print("Top 5 Players by Usage Rate (USG%):")
print(avg_usg.head(5))


Top 5 Players by Usage Rate (USG%):
player
Punter Kevin       13.644535
Leday Zach         11.260251
Lessort Mathias     9.040352
Exum Dante          8.773679
Nunnally James      6.643273
Name: usage_rate, dtype: float64


In [7]:
# snippet from points_per_possession.ipynb

# points per possession (PPP)
# scoring efficiency per possession used, highlighting players who maximize points from their offensive opportunities.
# formula: PPP = Points / (FGA + 0.44 * FTA + TO)
players['field_goals_attempted'] = players['two_points_attempted'] + players['three_points_attempted']
players['possessions_used'] = players['field_goals_attempted'] + 0.44 * players['free_throws_attempted'] + players['turnovers']
players['points_per_possession'] = players['points'] / players['possessions_used']


# handle division by zero (players with no possessions used)
players['points_per_possession'] = players['points_per_possession'].fillna(0).replace([float('inf'), -float('inf')], 0)


# average PPP per player
avg_PPP = players.groupby('player')['points_per_possession'].mean().sort_values(ascending=False)


print("Top 5 Players by Points per Possession (PPP):")
print(avg_PPP.head(5))

Top 5 Players by Points per Possession (PPP):
player
Lessort Mathias    1.232044
Exum Dante         1.146356
Punter Kevin       1.126225
Smailagic Alen     1.123254
Leday Zach         1.062734
Name: points_per_possession, dtype: float64


In [8]:
# snippet from rebound_rate.ipynb

# calculate Rebound Rate
# formula: Rebound Rate = 100 * (Player Total Rebounds / Team Total Rebounds) * (Player Total Seconds / game_seconds)
# Rebound Rate (REB%): Percentage of team rebounds secured, adjusted for playing time
# approximates rebounding impact using team totals as a proxy for available rebounds.

players_2 = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
team_2 = df[df['player'] == 'PARTIZAN MOZZART BET BELGRADE'].copy()

# merge team stats to players_2
players_2 = players_2.merge(team_2[['game', 'total_rebounds']], on='game', suffixes=('', '_team'))

# reset index to avoid duplicate labels
players_2 = players_2.reset_index(drop=True)

# calculate Rebound Rate
game_seconds = 2400  # 40 minutes * 60 seconds
players_2['rebound_rate'] = 100 * (players_2['total_rebounds'] / players_2['total_rebounds_team']) * (players_2['total_seconds'] / game_seconds)

# handle division by zero (players with no team stats)
players_2['rebound_rate'] = players_2['rebound_rate'].fillna(0).replace([float('inf'), -float('inf')], 0)

# average Rebound Rate per player
avg_rr = players_2.groupby('player')['rebound_rate'].mean().sort_values(ascending=False)

print("Top 5 Players by Rebound Rate:") 
print(avg_rr.head(5))


Top 5 Players by Rebound Rate:
player
Lessort Mathias    16.606726
Leday Zach         12.978675
Punter Kevin        5.821043
Nunnally James      4.828153
Exum Dante          4.804690
Name: rebound_rate, dtype: float64


In [9]:
# Display the top 5 players for each metric
print("Top 5 Players by True Shooting Percentage (TS%):")
print(avg_ts.head(5))
print("\nTop 5 Players by Points per 32 Minutes (PP32):")
print(avg_points_per_32.head(5))
print("\nTop 5 Players by Assist-to-Turnover Ratio (AST/TO):")
print(avg_ast_to.head(5))
print("\nTop 5 Players by Usage Rate (USG%):")
print(avg_usg.head(5))
print("\nTop 5 Players by Points per Possession (PPP):")
print(avg_PPP.head(5))
print("\nTop 5 Players by Rebound Rate (REB%):")
print(avg_rr.head(5))

Top 5 Players by True Shooting Percentage (TS%):
player
Lessort Mathias    0.721486
Exum Dante         0.671916
Punter Kevin       0.623184
Smailagic Alen     0.619905
Leday Zach         0.587875
Name: ts_percentage, dtype: float64

Top 5 Players by Points per 32 Minutes (PP32):
player
Punter Kevin        18.341875
Exum Dante          18.070395
Smailagic Alen      17.346076
Lessort Mathias     12.758641
Avramovic Aleksa    12.523459
Name: points_per_32, dtype: float64

Top 5 Players by Assist-to-Turnover Ratio (AST/TO):
player
Exum Dante          1.857456
Punter Kevin        1.844144
Nunnally James      1.401316
Madar Yam           1.391026
Avramovic Aleksa    1.201389
Name: ast_to_ratio, dtype: float64

Top 5 Players by Usage Rate (USG%):
player
Punter Kevin       13.644535
Leday Zach         11.260251
Lessort Mathias     9.040352
Exum Dante          8.773679
Nunnally James      6.643273
Name: usage_rate, dtype: float64

Top 5 Players by Points per Possession (PPP):
player
Lessort Mat

In [10]:
# summary table
summary_data = {
    'TS%': avg_ts.head(5).round(3),
    'PP32': avg_points_per_32.head(5).round(2),
    'AST/TO': avg_ast_to.head(5).round(2),
    'USG%': avg_usg.head(5).round(2),
    'PPP': avg_PPP.head(5).round(3),
    'REB%': avg_rr.head(5).round(2)
}
summary_df = pd.DataFrame(summary_data)
 

print("\nSummary of Top Performers Across Metrics:")
print(summary_df)


Summary of Top Performers Across Metrics:
                    TS%   PP32  AST/TO   USG%    PPP   REB%
player                                                     
Avramovic Aleksa    NaN  12.52    1.20    NaN    NaN    NaN
Exum Dante        0.672  18.07    1.86   8.77  1.146   4.80
Leday Zach        0.588    NaN     NaN  11.26  1.063  12.98
Lessort Mathias   0.721  12.76     NaN   9.04  1.232  16.61
Madar Yam           NaN    NaN    1.39    NaN    NaN    NaN
Nunnally James      NaN    NaN    1.40   6.64    NaN   4.83
Punter Kevin      0.623  18.34    1.84  13.64  1.126   5.82
Smailagic Alen    0.620  17.35     NaN    NaN  1.123    NaN


## Deeper Analysis: Key Players and Strategic Insights

### Identify Key Players and Their Strengths  

#### Lessort Mathias  
**Metrics:**  
- TS%: 0.721486 (1st)  
- PP32: 12.758641 (4th)  
- USG%: 9.040352 (3rd)  
- PPP: 1.232044 (1st)  
- REB%: 16.606726 (1st)  

**Analysis:**  
- **Scoring Efficiency:** Lessort leads in TS% (0.721486) and PPP (1.232044), indicating he’s the most efficient scorer on the team. He maximizes points per possession and shoots effectively across all types of shots (2-pointers, 3-pointers, and free throws).  
- **Rebounding:** His REB% of 16.606726 is the highest on the team, showing he’s a dominant rebounder, likely playing a big man role (e.g., center or power forward). This makes him crucial for securing second-chance opportunities and defending the paint.  
- **Usage:** A USG% of 9.040352 (3rd) suggests he’s a significant part of the offense but not the primary ball-handler, allowing him to focus on high-efficiency plays like post-ups, putbacks, and free throws.  
- **Scoring Rate:** His PP32 of 12.758641 is solid but not elite, indicating he’s not a high-volume scorer but makes the most of his opportunities.  
- **Role:** Lessort is likely a highly efficient big man who anchors the team’s interior presence. His ability to score efficiently, rebound at a high rate, and contribute to the offense without needing the ball too often makes him a cornerstone of the team’s frontcourt.

#### Punter Kevin  
**Metrics:**  
- TS%: 0.623184 (3rd)  
- PP32: 18.341875 (1st)  
- AST/TO: 1.844144 (2nd)  
- USG%: 13.644535 (1st)  
- PPP: 1.126225 (3rd)  
- REB%: 5.821043 (3rd)  

**Analysis:**  
- **Scoring Rate:** Punter leads in PP32 (18.341875), showing he’s the team’s most prolific scorer when normalized for playing time. He’s likely a primary scoring option, possibly a guard or wing.  
- **Usage:** His USG% of 13.644535 is the highest on the team, indicating he’s the focal point of the offense, handling the ball frequently and taking on a large share of the team’s possessions.  
- **Playmaking:** An AST/TO ratio of 1.844144 (2nd) shows he’s an effective playmaker, balancing assists with minimal turnovers. This suggests he’s not just a scorer but also a creator for his teammates.  
- **Scoring Efficiency:** His TS% (0.623184) and PPP (1.126225) are strong, ranking 3rd in both, meaning he’s efficient despite his high usage.  
- **Rebounding:** A REB% of 5.821043 is decent for a guard or wing, showing he contributes on the boards, though it’s not his primary role.  
- **Role:** Punter is likely the team’s primary scoring guard (e.g., shooting guard or point guard), serving as the offensive engine. His high usage, scoring rate, and playmaking ability make him a go-to player in clutch situations, while his efficiency ensures he’s not just a volume shooter.

#### Exum Dante  
**Metrics:**  
- TS%: 0.671916 (2nd)  
- PP32: 18.070395 (2nd)  
- AST/TO: 1.857456 (1st)  
- USG%: 8.773679 (4th)  
- PPP: 1.146356 (2nd)  
- REB%: 4.804690 (5th)  

**Analysis:**  
- **Scoring Efficiency:** Exum ranks 2nd in TS% (0.671916) and PPP (1.146356), showing he’s highly efficient at converting scoring opportunities.  
- **Scoring Rate:** His PP32 of 18.070395 (2nd) indicates he’s a high-volume scorer, nearly matching Punter’s output.  
- **Playmaking:** Leading the team in AST/TO (1.857456) suggests he’s the best at facilitating the offense without turning the ball over, likely playing as a point guard or combo guard.  
- **Usage:** A USG% of 8.773679 (4th) is moderate, meaning he’s involved in the offense but not as heavily as Punter or Leday, allowing him to focus on efficient plays.  
- **Rebounding:** His REB% of 4.804690 is the lowest among the top 5, which is expected for a guard focused on scoring and playmaking.  
- **Role:** Exum is likely a highly efficient guard (possibly a point guard) who balances scoring and playmaking. His ability to score efficiently, create for others, and maintain a high scoring rate makes him a versatile backcourt player.

#### Leday Zach  
**Metrics:**  
- TS%: 0.587875 (5th)  
- USG%: 11.260251 (2nd)  
- PPP: 1.062734 (5th)  
- REB%: 12.978675 (2nd)  

**Analysis:**  
- **Usage:** Leday’s USG% of 11.260251 (2nd) indicates he’s a significant part of the offense, taking on a large share of possessions.  
- **Rebounding:** His REB% of 12.978675 (2nd) is strong, suggesting he’s a forward or big man who contributes heavily on the boards.  
- **Scoring Efficiency:** His TS% (0.587875) and PPP (1.062734) are the lowest among the top 5, indicating he’s less efficient than players like Lessort or Exum. He might be taking tougher shots or struggling with consistency.  
- **Role:** Leday is likely a forward (e.g., power forward) who plays a significant role in the offense and rebounding. While he’s heavily involved, his lower efficiency suggests he might benefit from better shot selection or more support from playmakers.

#### Smailagic Alen  
**Metrics:**  
- TS%: 0.619905 (4th)  
- PP32: 17.346076 (3rd)  
- PPP: 1.123254 (4th)  

**Analysis:**  
- **Scoring Rate:** Smailagic’s PP32 of 17.346076 (3rd) shows he’s a high-volume scorer, close to Punter and Exum.  
- **Scoring Efficiency:** His TS% (0.619905) and PPP (1.123254) are solid, ranking 4th in both, indicating he’s efficient but not elite.  
- **Other Metrics:** He doesn’t appear in the top 5 for USG%, AST/TO, or REB%, suggesting he’s primarily a scorer rather than a playmaker or rebounder.  
- **Role:** Smailagic is likely a scoring forward or wing who provides offensive firepower off the bench or as a starter. His focus on scoring makes him a valuable offensive weapon, though he may not contribute as much in other areas.

#### Nunnally James  
**Metrics:**  
- AST/TO: 1.401316 (3rd)  
- USG%: 6.643273 (5th)  
- REB%: 4.828153 (4th)  

**Analysis:**  
- **Playmaking:** Nunnally’s AST/TO of 1.401316 (3rd) indicates he’s a reliable playmaker, likely a wing or guard who can handle the ball.  
- **Usage:** His USG% of 6.643273 is the lowest among the top 5, suggesting he’s not a primary offensive option.  
- **Rebounding:** A REB% of 4.828153 is decent for a wing, showing he contributes on the boards.  
- **Role:** Nunnally is likely a role player (e.g., small forward or shooting guard) who provides playmaking and rebounding support. His lower usage suggests he’s a complementary piece rather than a focal point.

#### Avramovic Aleksa  
**Metrics:**  
- PP32: 12.523459 (5th)  
- AST/TO: 1.201389 (5th)  

**Analysis:**  
- **Scoring Rate:** Avramovic’s PP32 of 12.523459 (5th) is decent but not elite, indicating he’s a moderate scorer.  
- **Playmaking:** His AST/TO of 1.201389 (5th) shows he can facilitate but isn’t as efficient as Exum or Punter.  
- **Role:** Avramovic is likely a backup guard who provides scoring and playmaking off the bench. His metrics suggest he’s a solid contributor but not a star.

### Identify Key Players for Partizan Mozzart Bet Belgrade  
Based on the analysis, the following players stand out as key contributors in the 2022/23 season:

#### Lessort Mathias (Big Man)  
- **Strengths:** Elite scoring efficiency (TS%, PPP), dominant rebounding (REB%), and solid usage (USG%).  
- **Impact:** Lessort is the team’s most efficient scorer and best rebounder, making him a cornerstone of the frontcourt. His ability to score efficiently in the paint and secure rebounds is crucial for both offense and defense.  
- **Strategic Role:** He should be a focal point in the paint, receiving touches for high-percentage shots and anchoring the team’s rebounding efforts. His efficiency makes him a reliable option in crunch time.

#### Punter Kevin (Primary Guard)  
- **Strengths:** High scoring rate (PP32), high usage (USG%), strong playmaking (AST/TO), and good efficiency (TS%, PPP).  
- **Impact:** Punter is the team’s offensive engine, leading in usage and scoring rate while maintaining efficiency and playmaking ability. He’s likely the go-to player in critical moments.  
- **Strategic Role:** Punter should continue to be the primary ball-handler and scorer, with the freedom to take over games. His playmaking ability also allows him to set up teammates, making him a dual threat.

#### Exum Dante (Versatile Guard)  
- **Strengths:** High scoring efficiency (TS%, PPP), high scoring rate (PP32), and elite playmaking (AST/TO).  
- **Impact:** Exum complements Punter in the backcourt, providing efficient scoring and the best playmaking on the team. His versatility allows him to score or facilitate as needed.  
- **Strategic Role:** Exum should share ball-handling duties with Punter, focusing on creating for others while taking advantage of his scoring efficiency. He’s a key piece for maintaining offensive flow.

#### Leday Zach (Forward)  
- **Strengths:** High usage (USG%) and strong rebounding (REB%).  
- **Impact:** Leday is a significant part of the offense and a strong rebounder, but his lower efficiency (TS%, PPP) suggests he might be taking on too many difficult shots.  
- **Strategic Role:** Leday should focus on higher-percentage shots, possibly by working off plays created by Punter and Exum. His rebounding makes him valuable in the frontcourt alongside Lessort.

### Team Insights and Strategic Implications  

#### Offensive Balance  
- **Punter Kevin** and **Exum Dante** form a dynamic backcourt, combining high usage, scoring, and playmaking. They can drive the offense, create their own shots, and set up teammates.  
- **Lessort Mathias** provides an efficient scoring option in the paint, complementing the guards’ perimeter play. His high TS% and PPP make him a reliable target for lobs, post-ups, and second-chance points.  
- **Leday Zach**’s high usage is valuable, but his lower efficiency suggests he might benefit from better shot selection or more off-ball opportunities created by the guards.

#### Rebounding Strength  
- **Lessort Mathias** and **Leday Zach** anchor the team’s rebounding, with REB% values of 16.606726 and 12.978675, respectively. This gives Partizan a strong presence on the boards, crucial for controlling the game’s tempo and securing extra possessions.  
- The guards (**Punter**, **Exum**, **Nunnally**) contribute modestly to rebounding, which is a bonus for their positions.

#### Playmaking  
- **Exum Dante** and **Punter Kevin** lead in AST/TO, ensuring the team has reliable facilitators. This allows for a balanced offense where the ball moves efficiently, creating opportunities for players like Lessort and Smailagic.

#### Areas for Improvement  
- **Leday Zach’s Efficiency:** His high usage (11.260251) paired with lower TS% (0.587875) and PPP (1.062734) suggests he might be forcing shots. The team could design plays to get him easier looks, such as pick-and-rolls with Punter or Exum.  
- **Rebounding from Guards:** While Punter, Exum, and Nunnally contribute to rebounding, the team might benefit from more effort on the boards from the backcourt to support Lessort and Leday.  
- **Depth Scoring:** **Smailagic Alen** and **Avramovic Aleksa** provide scoring off the bench, but their efficiency (TS%, PPP) isn’t as high as the starters. Developing their consistency could help maintain offensive output when the starters rest.

### Recommendations  

- **Maximize Lessort’s Touches in the Paint:** Lessort’s elite efficiency (TS%, PPP) and rebounding (REB%) make him a key weapon. The team should prioritize getting him the ball in high-percentage situations, such as post-ups, lobs, and offensive rebounds.  
- **Leverage Punter and Exum’s Versatility:** Punter and Exum should continue to lead the offense, balancing their scoring with playmaking. Stagger their minutes to ensure one of them is always on the court to maintain offensive flow.  
- **Improve Leday’s Shot Selection:** Leday’s high usage is valuable, but his efficiency needs improvement. The team could run more plays to get him open looks, such as screens or cuts, rather than relying on him to create his own shots.  
- **Encourage Guard Rebounding:** Punter, Exum, and Nunnally should be encouraged to crash the boards more often, especially on defense, to support Lessort and Leday. This could help the team dominate rebounding margins.  
- **Develop Bench Scoring:** Smailagic Alen and Avramovic Aleksa are solid bench scorers, but their efficiency could improve. The team might focus on integrating them into the offense with set plays to boost their confidence and consistency.